In [31]:
import re
import os
import warnings
import datetime

# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymystem3 import Mystem
from nltk.corpus import stopwords

In [30]:
for file in os.listdir('Telegram prep'):
    df = pd.read_parquet(f'Telegram prep/{file}')
    df['date'] = pd.to_datetime(df['date'].astype('str').map(lambda x: x[:-6])) + datetime.timedelta(hours=3)
    df.to_parquet(f'Telegram prep/{file}')
    print(file)

ak47pfl_new.parquet
antiskrepa_new.parquet
banksta_new.parquet
bitkogan_new.parquet
cbonds_new.parquet
cbrstocks_new.parquet
divForever_new.parquet
DividendNews100_new.parquet
economika_new.parquet
economylive_new.parquet
energytodaygroup_new.parquet
FatCat18_new.parquet
finascop_new.parquet
gazmyaso_new.parquet
gazpromneft_official_new.parquet
gazprom_new.parquet
information_disclosure_new.parquet
investingcorp_new.parquet
kotelviry_new.parquet
kremlin_mother_expert_new.parquet
marketoverview_new.parquet
markettwits_new.parquet
mislinemisli_new.parquet
moscowexchangeofficial_new.parquet
nebrexnya_new.parquet
nedvizhna24_new.parquet
papagaz_new.parquet
rbc_news_new.parquet
rian_ru_new.parquet
rt_russian_new.parquet
russianmacro_new.parquet
smartlabnews_new.parquet
StockNews100_new.parquet


In [52]:
for file in os.listdir('Telegram prep'):
    df = pd.read_parquet(f'Telegram prep/{file}')
    df = df[~df['message'].str.contains('реклама|подписаться|канал|youtube|скидка|бесплатно|рекомендация', regex=True)]
    df.to_parquet(f'Telegram prep/{file}', index=False)
    print(file)

ak47pfl_new.parquet
antiskrepa_new.parquet
banksta_new.parquet
bitkogan_new.parquet
cbonds_new.parquet
cbrstocks_new.parquet
divForever_new.parquet
DividendNews100_new.parquet
economika_new.parquet
economylive_new.parquet
energytodaygroup_new.parquet
FatCat18_new.parquet
finascop_new.parquet
gazmyaso_new.parquet
gazpromneft_official_new.parquet
gazprom_new.parquet
information_disclosure_new.parquet
investingcorp_new.parquet
kotelviry_new.parquet
kremlin_mother_expert_new.parquet
marketoverview_new.parquet
markettwits_new.parquet
mislinemisli_new.parquet
moscowexchangeofficial_new.parquet
nebrexnya_new.parquet
nedvizhna24_new.parquet
papagaz_new.parquet
rbc_news_new.parquet
rian_ru_new.parquet
rt_russian_new.parquet
russianmacro_new.parquet
smartlabnews_new.parquet
StockNews100_new.parquet


## Небольшая работа с файлами с ценами акций

In [2]:
companies = ['GAZP', 'SBER', 'VTBR', 'ROSN', 'LKOH', 'YNDX', 'NVTK', 'GMKN', 'AFLT', 'TCSG']

In [38]:
df = pd.read_parquet('D:/Жора/Data/targets/GAZP.parquet')

In [39]:
df

open        high         low       close      vol  \
0        192.550003  194.250000  191.000000  194.000000  1489521   
1        194.139999  194.300003  193.570007  193.570007   790275   
2        193.610001  193.610001  193.149994  193.309998   561725   
3        193.309998  193.399994  193.100006  193.300003   299942   
4        193.229996  193.699997  193.229996  193.649994   415095   
...             ...         ...         ...         ...      ...   
1967294  176.050000  176.050000  176.000000  176.000000    11100   
1967295  176.030000  176.030000  176.010000  176.030000     4400   
1967296  176.030000  176.090000  176.030000  176.090000    35340   
1967297  176.080000  176.090000  176.060000  176.070000     5100   
1967298  176.070000  176.070000  176.060000  176.060000      450   

                       date  
0       2010-01-11 10:31:00  
1       2010-01-11 10:32:00  
2       2010-01-11 10:33:00  
3       2010-01-11 10:34:00  
4       2010-01-11 10:35:00  
...                     ...  
1967294 2023-08-11 20:15:00  
1967295 2023-08-11 20:16:00  
1967296 2023-08-11 20:17:00  
1967297 2023-08-11 20:18:00  
1967298 2023-08-11 20:19:00  

[1967299 rows x 6 columns]

In [5]:
files = os.listdir('D:/Жора/Data/targets')
files = [file for file in files if 'csv' in file]

In [37]:
for ticker, file in zip(sorted(companies), files):
    parquet = pd.read_parquet(f'D:/Жора/Data/targets/{ticker}.parquet')
    try:
        csv = pd.read_csv(f'D:/Жора/Data/targets/{file}')
        csv['date'] = pd.to_datetime(csv['<DATE>'].apply(str) + csv['<TIME>'].apply(str),
                        format='%Y%m%d%H%M%S')
    except KeyError:
        csv = pd.read_csv(f'D:/Жора/Data/targets/{file}', sep=';')
        csv['date'] = pd.to_datetime(csv['<DATE>'].apply(str) + csv['<TIME>'].apply(str),
                        format='%Y%m%d%H%M%S')
    csv.rename(columns={'<OPEN>': 'open', '<HIGH>': 'high', '<LOW>': 'low',
                   '<CLOSE>': 'close', '<VOL>': 'vol'}, inplace=True)
    csv.drop(['<TICKER>', '<PER>', '<DATE>', '<TIME>'], axis=1, inplace=True)
    
    parquet = pd.concat([parquet, csv])
    parquet.reset_index(inplace=True, drop=True)
    parquet.to_parquet(f'D:/Жора/Data/targets/{ticker}.parquet')

## Продолжение работы с текстами

In [45]:
first_dates = pd.DataFrame(columns=['company', 'date'])
first_dates['company'] = [None] * 10
first_dates['date'] = [None] * 10
for i, ticker in enumerate(companies):
    price = pd.read_parquet(f'D:/Жора/Data/targets/{ticker}.parquet')['date']
    date = pd.to_datetime(price[0])
    first_dates.iloc[i,:] = [ticker, date]

In [47]:
first_dates.to_csv('D:/Жора/Data/first_dates_new.csv', index=False)

In [26]:
import emoji

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [1]:
for file in files:
    print(f'file {file} in work')
    warnings.filterwarnings('ignore')
    df = pd.read_parquet(f'Telegram prep/{file[:-4]}parquet')
    df['message'] = df['message'].str.replace('\n', ' ').str.replace('\r', ' ')
    df['message'] = df['message'].apply(give_emoji_free_text)
    warnings.filterwarnings('default')
    df.to_parquet(f'Telegram prep/{file[:-4]}parquet', index=False)
    print(f'{c} из {len(files)} готов')
    c += 1

In [56]:
for file in os.listdir('Telegram prep'):
    new_file = file.replace('new', 'raw')
    os.rename(f'Telegram prep/{file}', f'Telegram prep/{new_file}')

In [62]:
mystem = Mystem()
swords = set(stopwords.words('russian'))

def preprocess_tg(df):
    try:
        n_rows = df.shape[0]
        c = 0
        cols = ['message']
        for col in cols:
            df[col] = df[col].str.replace('\n', '')
            df[col] = df[col].str.replace('\W+', ' ', regex=True)
            df[col] = df[col].str.lower()
            while c < n_rows:
                res = []
                doc = []
                large_str = ' '.join([txt + ' splitter ' for txt in df[col][c:c+1000]])
                large_str = mystem.lemmatize(large_str)

                for word in large_str:
                    if word.strip() != '' and word not in swords:
                        if word == 'splitter':
                            if len(doc) == 0:
                                doc.append('')
                            res.append(doc)
                            doc = []
                        else:
                            doc.append(word)
                del large_str
                del doc
                res = [' '.join(lst) for lst in res]
                df[col][c:c+1000] = res
                c += 1000
            print(f'{col} ready')
        return df
    except KeyboardInterrupt:
        raise
    except Exception as ex:
        print(ex)
        return None

In [65]:
files = os.listdir('Telegram prep')
if '.DS_Store' in files:
    files.remove('.DS_Store')
c = 1

for file in files:
    print(f'file {file} in work')
    warnings.simplefilter("ignore")
    df = pd.read_parquet(f'Telegram prep/{file}')
    df = preprocess_tg(df)
    df.to_parquet(f'Telegram prep/{file[:-4]}_lemm.parquet', index=False)
    warnings.simplefilter("default")
    print(f'{c} из {len(files)} готов')
    c += 1

file ak47pfl_raw.parquet in work
message ready
1 из 33 готов
file antiskrepa_raw.parquet in work
message ready
2 из 33 готов
file banksta_raw.parquet in work
message ready
3 из 33 готов
file bitkogan_raw.parquet in work
message ready
4 из 33 готов
file cbonds_raw.parquet in work
message ready
5 из 33 готов
file cbrstocks_raw.parquet in work
message ready
6 из 33 готов
file divForever_raw.parquet in work
message ready
7 из 33 готов
file DividendNews100_raw.parquet in work
message ready
8 из 33 готов
file economika_raw.parquet in work
message ready
9 из 33 готов
file economylive_raw.parquet in work
message ready
10 из 33 готов
file energytodaygroup_raw.parquet in work
message ready
11 из 33 готов
file FatCat18_raw.parquet in work
message ready
12 из 33 готов
file finascop_raw.parquet in work
message ready
13 из 33 готов
file gazmyaso_raw.parquet in work
message ready
14 из 33 готов
file gazpromneft_official_raw.parquet in work
message ready
15 из 33 готов
file gazprom_raw.parquet in work

### Обрабатываем даты, чтобы они были стандартные

In [10]:
def telegram_to_standard_date():
    files = os.listdir('Telegram prep')
    if '.DS_Store' in files:
        files.remove('.DS_Store')
    files = [f for f in files if 'parquet' in f]

    for file in files:
        print(file)
        df = pd.read_parquet(f'Telegram prep/{file}')
        df['date'] = pd.to_datetime(df['date'].map(lambda x: str(x)[:-6]))
        df.to_parquet(f'Telegram prep/{file}', index=False)

In [14]:
telegram_to_standard_date()

ak47pfl_messages.parquet
antiskrepa_messages.parquet
banksta_messages.parquet
bitkogan_messages.parquet
cbonds_messages.parquet
cbrstocks_messages.parquet
divForever_messages.parquet
DividendNews100_messages.parquet
economika_messages.parquet
economylive_messages.parquet
energytodaygroup_messages.parquet
FatCat18_messages.parquet
finascop_messages.parquet
gazmyaso_messages.parquet
gazpromneft_official_messages.parquet
gazprom_messages.parquet
information_disclosure_messages.parquet
investingcorp_messages.parquet
kotelviry_messages.parquet
kremlin_mother_expert_messages.parquet
marketoverview_messages.parquet
markettwits_messages.parquet
mislinemisli_messages.parquet
moscowexchangeofficial_messages.parquet
nebrexnya_messages.parquet
nedvizhna24_messages.parquet
papagaz_messages.parquet
rbc_news_messages.parquet
rian_ru_messages.parquet
rt_russian_messages.parquet
russianmacro_messages.parquet
smartlabnews_messages.parquet
smartlab_messages.parquet
StockNews100_messages.parquet


### Собираем данные под обучение

In [68]:
files = os.listdir('Telegram prep')
files = [f for f in files if 'lemm' in f]
for file in files:
    df = pd.read_parquet(f'Telegram prep/{file}')
    df = df[['date', 'message', 'views', 'forwards'] + companies].copy()
    df = df[df[companies].any(axis=1)].copy()
    df.to_parquet(f'Telegram prep/{file}', index=False)

In [102]:
files = os.listdir('Telegram prep')
files = [f for f in files if 'lemm' in f]
agg_df = pd.DataFrame()
for file in files:
    df = pd.read_parquet(f'Telegram prep/{file}')
    df['source'] = file[:-13]
    agg_df = pd.concat([agg_df, df])
    
agg_df.drop_duplicates('message', inplace=True)
agg_df.to_parquet('data_lemm.parquet', index=False)

In [114]:
files = os.listdir('Telegram prep')
files = [f for f in files if 'raw' in f]

In [123]:
files.remove(file)

In [128]:
agg_df = pd.DataFrame()
for file in files:
    df = pd.read_parquet(f'Telegram prep/{file}')
    df.drop(['fwd_from'], axis=1, inplace=True)
    df.drop_duplicates('message', inplace=True)
    df['source'] = file[:-12]
    agg_df = pd.concat([agg_df, df])
agg_df.drop_duplicates('message', inplace=True)
agg_df.to_parquet('data_raw.parquet', index=False)

In [132]:
df_lemm = pd.read_parquet('data_lemm.parquet')
df_raw = pd.read_parquet('data_raw.parquet')

In [134]:
df_raw

date                                            message  \
0      2023-07-24 19:03:02  Кэрри-трейд: что нужно знать инвестору. #рдв_о...   
1      2023-07-24 18:00:41       #опрос Рейтинг акций коммьюнити РДВ @AK47pfl   
2      2023-07-24 16:02:08  ТОЛЬКО ДВА ДНЯ до среды открыт доступ к вступл...   
3      2023-07-24 10:01:07  Рейтинг акций коммьюнити РДВ *Какой акции боль...   
4      2023-07-24 09:30:55  Главное к открытию понедельника (24.07): #бриф...   
...                    ...                                                ...   
982162 2019-10-20 19:35:44  🇹🇷#турция #нато Марин Ле Пен призвала приостан...   
982163 2019-10-20 19:31:52  #MAIL #сми #регулирование #горелкин ПРАВИТЕЛЬС...   
982164 2019-10-20 19:12:30  #россия #европа #бизнес Медведев: РФ стало тру...   
982165 2019-10-20 19:09:09  БРИТАНСКИЕ ДЕПУТАТЫ В СУББОТУ ОДОБРИЛИ ПОПРАВК...   
982166 2019-10-20 18:39:50  #россия #ОПЕК Россия в сентябре не полностью в...   

          views  forwards        source  
0       16655.0      44.0       ak47pfl  
1       15241.0      13.0       ak47pfl  
2       21834.0       8.0       ak47pfl  
3       19476.0       3.0       ak47pfl  
4       24897.0      11.0       ak47pfl  
...         ...       ...           ...  
982162     37.0       0.0  StockNews100  
982163     35.0       0.0  StockNews100  
982164     54.0       1.0  StockNews100  
982165     43.0       0.0  StockNews100  
982166     44.0       0.0  StockNews100  

[982167 rows x 5 columns]

In [146]:
df_lemm.drop_duplicates(['date', 'views', 'forwards', 'source'], inplace=True)
df_raw.drop_duplicates(['date', 'views', 'forwards', 'source'], inplace=True)

In [149]:
final = df_lemm.merge(df_raw, how='left', on=['date', 'views', 'forwards', 'source'])

In [154]:
# final.rename(columns={'message_x': 'message_lemm', 'message_y': 'message_raw'}, inplace=True)
final = final[['date', 'message_lemm', 'message_raw', 'views', 'forwards'] + companies].copy()

In [156]:
final.to_parquet('data.parquet')

### Колонки для определения принадлежности новости к компании

In [70]:
companies = {
    'ALRS': 'alrosa-ao', 'AFLT': 'aeroflot', 'VTBR': 'vtb', 'GAZP' : 'gazprom', 'GMKN': 'nornickel-gmk', 'LSRG': 'lsr', 
    'DSKY': 'detsky-mir', 'IRAO': 'inter-rao-ao', 'LKOH': 'lukoil', 'MAGN': 'mmk', 'MOEX': 'moscowexchange', 'CBOM': 'oao-mkb-ao', 
    'MTSS': 'mts', 
    'ROSN': 'rosneft', 'NLMK': 'nlmk-ao', 
    'NVTK': 'novatek', 
    'PIKK': 'pik', 
    'PLZL': 'polus-zoloto', 'RTKM': 'rostelecom', 
    'RUAL': 'united-company-rusal-plc', 
    'HYDR': 'rusgidro', 'SBER': 'sberbank', 'CHMF': 'severstal-ao', 
    'AFKS': 'afk-sistema', 'SNGS': 'surgut', 'TATN': 'tatneft-3', 'TCSG': 'gdr-tcs-group-holding-ord-shs_tcsg', 'TRNFP': 'transneft-pref', 
    'POGR': 'petropavlovsk-plc_pogr', 
    'PHOR': 'phosagro-ao', 
    'FEES': 'fsk-ees', 'HHRU': 'adr-headhunter-group-ord-shs_hhru', 'YNDX': 'pllc-yandex-n-v'
}

In [71]:
for_regex = {
    'ALRS': ['алроса', 'alrs', 'алмазы анабара', 'севералмаз', 'горнорудное общество катока',
            'мирнинский гок', 'водораздельный галечник', 'иреляхский россыпь', 'мирнинский горно обогатительный комбинат',
            'айхальский горно обогатительный комбинат', 'удачнинский горно обогатительный комбинат',
            'нюрбинский горно обогатительный комбинат'],
    'AFLT': ['аэрофлот', 'aflt', 'aeroflot', 'авиакомпания россия', 'авиакомпания победа'],
    'VTBR': ['втб', 'почта банк', 'транскредитбанк', 'vtb', 'банк москвы', 'мультикарта', 'галс девелопмент',
            'бм банк', 'согаз'], 
    'GAZP': ['газпром', 'gazp', 'северный поток', 'сила сибири', 'голубой поток', 'ямал европа',
             'ямал европа', 'ухта торжок', 'gazprom'], 
    'GMKN': ['норникел', 'норильский никель', 'gmkn', 'рудник октябрьский'],
    'LSRG': ['лср', 'lsrg'],
    'DSKY': ['детский мир', 'dsky'],
    'IRAO': ['интер рао', 'irao', 'мосэнергосбыт', 'петербуржский энергосбытовой компания', 'объединенный энергосбытовой компания',
            'тгк-11', 'томский генерация', 'башкирская генерирующая компания', 'молдавская грэс'],
    'LKOH': ['лукойл', 'lkoh', 'лангепасурайкогалымнефть', 'lukoil'], 
    'MAGN': ['ммк', 'магнитогорский металлургический комбинат', 'magn', 'mmk', 'магнитка'],
    'MOEX': ['пао московская биржа', 'moex', 'мосбиржа'],
    'CBOM': ['мкб', 'московский кредитный банк', 'cbom'],
    'MTSS': ['мтс', 'mtss', 'mts', 'мобильный телесистема'], 
    'ROSN': ['роснефть', 'rosn', 'самаранефтегаз', 'rosneft', 'роснефти'],
    'NLMK': ['nlmk', 'новолипецкий металлургический комбинат', 'нлмк'],
    'NVTK': ['новатэк', 'NVTK', 'novatek', 'ямал спг', 'арктикгаз', 'нортгаз', 'арктик спг', 'транснефтегаз'],
    'PIKK': ['PIKK', 'пик'],
    'PLZL': ['plzl', 'полюс золото', 'пао полюс', 'ао полюс', 'polyus'],
    'RTKM': ['ростелеком', 'rtkm', 'wink'], 
    'RUAL': ['русал', 'rual', 'rusal', 'uc rusal'],
    'HYDR': ['русгидро', 'hydr', 'якутскэнерго', 'камчатскэнерго', 'магаданэнерго', 'сахалинэнерго', 'передвижная энергетикa',
            'красноярскэнергосбыт', 'рэск'],
    'SBER': ['/^сбер$/', 'сбербанк', 'sber', 'сбермегамаркет', 'сбермаркет', 'сберздоровье'],
    'CHMF': ['северсталь', 'chmf', 'severstal'],
    'AFKS': ['афк', 'afks', 'афк система', 'мтс', 'mtss', 'mts', 'мобильные телесистемы', 'мтса', 'мтсом',
            'озон', 'агрохолдинг степь', 'медси', 'эталон'],
    'SNGS': ['сургутнефтегаз', 'sngs', 'surgutneftegas'],
    'TATN': ['tatn', 'татнефт', 'tatneft', 'tatnp'],
    'TCSG': ['tcsg', 'тинькофф', 'tinkoff'],
    'TRNFP': ['транснефт', 'trnfp', 'transneft'],
    'POGR': ['петропавловск золото', 'petropavlovsk plc', 'petropavlovsk', 'pogr'], 
    'PHOR': ['фосагро', 'phor', 'phosagro'],
    'FEES': ['фск еэс', 'fees', 'россети', 'федеральная сетевая компания единой энергетической системы'], 
    'HHRU': ['headhunter', 'hhru', 'зарплата ру'],
    'YNDX': ['яндекс', 'yandex', 'yndx']
}

In [72]:
comps = [key for key in companies.keys()]

In [76]:
add_company_tg(for_regex, comps)

ak47pfl_raw.par_lemm.parquet done
antiskrepa_raw.par_lemm.parquet done
banksta_raw.par_lemm.parquet done
bitkogan_raw.par_lemm.parquet done
cbonds_raw.par_lemm.parquet done
cbrstocks_raw.par_lemm.parquet done
divForever_raw.par_lemm.parquet done
DividendNews100_raw.par_lemm.parquet done
economika_raw.par_lemm.parquet done
economylive_raw.par_lemm.parquet done
energytodaygroup_raw.par_lemm.parquet done
FatCat18_raw.par_lemm.parquet done
finascop_raw.par_lemm.parquet done
gazmyaso_raw.par_lemm.parquet done
gazpromneft_official_raw.par_lemm.parquet done
gazprom_raw.par_lemm.parquet done
information_disclosure_raw.par_lemm.parquet done
investingcorp_raw.par_lemm.parquet done
kotelviry_raw.par_lemm.parquet done
kremlin_mother_expert_raw.par_lemm.parquet done
marketoverview_raw.par_lemm.parquet done
markettwits_raw.par_lemm.parquet done
mislinemisli_raw.par_lemm.parquet done
moscowexchangeofficial_raw.par_lemm.parquet done
nebrexnya_raw.par_lemm.parquet done
nedvizhna24_raw.par_lemm.parquet 